In [39]:
import numpy as np
import ROOT as r
import random
import os
import matplotlib.pyplot as plt
# -----------------------------------------------------------------
# Import ROOT libraries for MadGraph generated rootfiles.
# -----------------------------------------------------------------
r.gROOT.ProcessLine(".include /Users/isaac/Work/MG5_aMC_v2_8_3_2")
r.gInterpreter.Declare(
    '#include "/Users/isaac/Work/MG5_aMC_v2_8_3_2/ExRootAnalysis/\
ExRootAnalysis/ExRootTreeReader.h"'
)
r.gInterpreter.Declare(
    '#include "/Users/isaac/Work/MG5_aMC_v2_8_3_2/\
ExRootAnalysis/ExRootAnalysis/ExRootClasses.h"'
)
r.gInterpreter.Declare(
    '#include "/Users/isaac/Work/MG5_aMC_v2_8_3_2/ExRootAnalysis/\
ExRootAnalysis/ExRootLHEFReader.h"'
)
r.gSystem.Load(
    "/Users/isaac/Work/MG5_aMC_v2_8_3_2/\
ExRootAnalysis/libExRootAnalysis.so"
)

# -----------------------------------------------------------------
# set up environment
# -----------------------------------------------------------------

os.environ["TERM"] = "linux"
random.seed(1)

In [40]:
luminosity = 1.5e04
event_path = "/Users/isaac/Work/MUonE/MG5_events/lepton2to3vec/Events/"
m_e = 5.11e-04
meter = 1
GeV = 1 / (1.97 * meter * 1.0e-16)
Radius = 0.05
angle_cut=0.001
L_min=0.02
L_max=0.145
x_sec=7.4e+08
chain = r.TChain("LHEF")
chain.Add("~/Work/MUonE/MG5_events/lepton2to3vec/Events/X_mass_0.01/unweighted_events.root")

1

In [20]:
def decay_rate(mass, g_e):
    alpha_e = (g_e ** 2) / (4 * np.pi)
    return (
        alpha_e
        * mass
        * (1 + 2 * ((m_e ** 2) / (mass ** 2)))
        * np.sqrt(1 - 4 * ((m_e ** 2) / (mass ** 2)))
        * GeV
        / 3
    )


def distance(momentum, mass, g_e):
    return momentum / (decay_rate(mass, g_e) * mass)


def probability(momentum, theta, mass, g_e):
    """This function calculates the probability for one
    single X particle to travel into the detectable region.
    open angle cuts is not applied here."""
    travel_distance = distance(momentum, mass, g_e)
    # Here loop over all the targets
    if L_max * np.tan(theta) <= Radius:
        L_prob = np.exp(-L_min / travel_distance) * (
            1.0 - np.exp((L_min - L_max) / travel_distance)
        )
    else:
        L_prob = 0.0
    return L_prob


def read_x_section(mass):
    """Read the cross section from madgraph banner file"""
    banner_file_subfolder = "X_mass_" + str(mass)
    banner_file_name = "X_mass_" + str(mass) + "_tag_1_banner.txt"
    banner_file_path = os.path.join(event_path, banner_file_subfolder, banner_file_name)
    with open(banner_file_path) as banner_file:
        for line in banner_file:
            if line.startswith("#  Integrated weight (pb)"):
                for x in line:
                    if x in ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]:
                        index = line.index(x)
                        break
                x_section_value = float(line[index:])
    return x_section_value


def get_sensitivity(mass, g_e, x_section):
    """Calculate the total number of signal events, given the probability for
    a single X particle to decay inside the volume.
    Open angle cuts is applied here."""
    rescaled_number = x_section * luminosity * (g_e ** 2)
    avg_probability = 0
    for i in range(len(X_list)):
        E = X_list[i][0]
        p = X_list[i][1]
        theta = X_list[i][2]
        open_angle = 2.0 * mass / E
        if open_angle >= angle_cut:
            avg_probability += probability(p, theta, mass, g_e)
    sensitivity = (avg_probability / 10000.0) * rescaled_number
    return sensitivity

In [22]:
X_list=[]
for event in chain:
    for particle in event.Particle:
        if particle.PID==103:
            E_X=particle.E
            p_X=np.sqrt(particle.E**2-particle.M**2)
            X_vector=r.TLorentzVector()
            X_vector.SetPtEtaPhiM(particle.PT, particle.Eta, particle.Phi, particle.M)
            theta_X=X_vector.Theta()
            X_list.append([E_X, p_X, theta_X])

In [24]:
g_e=2.0e-06
sensitivity=get_sensitivity(0.01,g_e,x_sec)
print(sensitivity)

2.0263599608057676


In [34]:
distance(1,0.01,2.0e-06)

18.567574851909203

In [37]:
probability(1,0.0,0.01,2.0e-06)

0.006702332660781945

In [58]:
p_list = []
for event in chain:
    for particle in event.Particle:
        if particle.PID==103:
            X_p = np.sqrt(particle.E**2-0.01**2)
            p_list.append(X_p)
p_list=np.array(p_list)
hist, bins = np.histogram(p_list)
logbins=np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))


AttributeError: 'Figure' object has no attribute 'xscale'

<Figure size 432x288 with 0 Axes>

In [52]:
bins

array([9.76778182e-02, 1.18743710e+01, 2.36510642e+01, 3.54277575e+01,
       4.72044507e+01, 5.89811439e+01, 7.07578371e+01, 8.25345303e+01,
       9.43112235e+01, 1.06087917e+02, 1.17864610e+02])

In [1]:
import numpy as np
